Example of looping through all comments, and storing info in a dataframe

In [1]:
#Grab packages

import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
#Connect to webpage

options = webdriver.ChromeOptions()
#options.add_argument('--headless')


options.binary_location = "/usr/bin/google-chrome"
chrome_driver_binary = "/usr/local/bin/chromedriver"
#options.binary_location = "C:/Program Files (x86)/Google/Chrome/Application/chrome.exe"
#chrome_driver_binary = "C:/Users/Alek/chromedriver"
driver = webdriver.Chrome(chrome_driver_binary, chrome_options=options)



/home/ravisolter/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: use options instead of chrome_options
  # This is added back by InteractiveShellApp.init_path()


In [3]:
# Logging in
driver.get("https://auth.voxmedia.com/login?return_to=https://www.blackwhitereadallover.com/")

element = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.CLASS_NAME, "p-text-input"))
    )

username = driver.find_element_by_xpath('//*[@id="login[username]"]')
username.clear()
username.send_keys("rsolter")


password = driver.find_element_by_xpath('//*[@id="login[password]"]')
password.clear()
password.send_keys("pass123!")


button = driver.find_element_by_xpath('//*[@id="auth"]/div/form/fieldset[3]/input')
button.click()

In [4]:
# Directing to one example article
element = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.CLASS_NAME, "c-global-header__logo-large"))
    )




In [5]:

### Insert loop here to visit relevant URLs

driver.get("https://www.blackwhitereadallover.com/2020/4/3/21206623/miralem-pjanic-marco-verratti-juventus-paris-saint-germain-2020-serie-a-summer-transfer-rumors#comments")

## Grab article title, author

blog_title = driver.find_element_by_xpath('//*[@id="content"]/article/div[1]/div[1]/div[2]/h1')
print(blog_title.text)

article_author = driver.find_element_by_xpath('//*[@id="content"]/article/div[1]/div[1]/div[3]/span[1]/span[1]/a/span')
print(article_author.text)

element = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.CLASS_NAME, "c-comments__list"))
    )

#Find number of comments

n_comments = driver.find_element_by_xpath('//*[@id="comments"]/div[1]/span/span')
n_comments = int(n_comments.text)


Report: PSG turns down Miralem Pjanic-Marco Verratti swap deal
Danny Penza


In [34]:
#Second Loop through all comments, taking title, content, and meta data (recs, author, etc)

from selenium.common.exceptions import NoSuchElementException

comments_df = pd.DataFrame()

for i in range(1,n_comments+1):
    
    xpath = '//*[@id="comments"]/div[2]/div['+str(i)+']'
     
    comment_title = driver.find_element_by_xpath(xpath + '/div[1]')
    comment_body = driver.find_element_by_xpath(xpath + '/div[2]')
    
    
    if driver.find_elements_by_xpath(xpath + '/div[4]'):
        comment_meta = driver.find_element_by_xpath(xpath + '/div[4]')
    else:
        comment_meta = driver.find_element_by_xpath(xpath + '/div[3]')

    comment = pd.DataFrame([comment_title.text,comment_body.text,comment_meta.text])
    comments_df = pd.concat([comments_df,comment.T])
    
    ;
   

In [35]:
comments_df.columns = ['Comment Title','Comment Body','Comment Meta']

# Adding article title and author to comments df
comments_df.insert(0, 'Article', blog_title.text)
comments_df.insert(1, 'Author', article_author.text)

comments_df.head()



,Article,Author,Comment Title,Comment Body,Comment Meta
0,Report: PSG turns down Miralem Pjanic-Marco Ve...,Danny Penza,,interesting video about how some stats should/...,"Posted by resko on Apr 3, 2020 | 6:37 AM rep..."
0,Report: PSG turns down Miralem Pjanic-Marco Ve...,Danny Penza,,Well that was a tough decision for PSG!,"Posted by Paolo C on Apr 3, 2020 | 6:42 AM r..."
0,Report: PSG turns down Miralem Pjanic-Marco Ve...,Danny Penza,,hmmm…………maybe if we added Khedira……….or Bernar...,"Posted by Ricardo 1 on Apr 3, 2020 | 10:44 A..."
0,Report: PSG turns down Miralem Pjanic-Marco Ve...,Danny Penza,,Interest chart here. Various strikers over/und...,"Posted by Paolo C on Apr 3, 2020 | 9:59 AM r..."
0,Report: PSG turns down Miralem Pjanic-Marco Ve...,Danny Penza,,Dybala is an amazing finisher\nso much this,"Posted by #finoallafine on Apr 5, 2020 | 11:..."


In [ ]:
# Add columns for Author


# Add columns for # of recommendations

In [36]:
comments_df.to_csv('comments.csv', index=False)


In [40]:
driver.quit()

In [60]:
# Getting Author

ts = 'Posted  by Paolo C  on Apr 3, 2020 | 6:42 AM reply rec (8) flag'

start = ts.find('by ')
print(start)
start2 = start+3
print(start2)

print(ts[start2:])

end = ts.find(' on')
print(end)
end2 = end-1
print(end2)

print(ts[start2:end2])


# Getting recommendations

recs = ts.find('rec (')
print(recs)

rec_start = recs+5
rec_end = recs+7
recN = ts[rec_start:rec_end]
print(recN)


def only_numerics(seq):
    seq_type= type(seq)
    return seq_type().join(filter(seq_type.isdigit, seq))

print(only_numerics(recN))


8
11
Paolo C  on Apr 3, 2020 | 6:42 AM reply rec (8) flag
19
18
Paolo C
51
8)
8


In [12]:
## The loop below works for handling NAs, but it's returning recs from only the first level of comments

from selenium.common.exceptions import NoSuchElementException

# Searching on recommend element specifically (handing NAs)

recs_df = pd.DataFrame()

for i in range(1,n_comments+1):
    
    try:
        recs_xpath = '/html/body/div[2]/main/article/div[5]/div[1]/section/div[2]/div['+str(i)+']/div[3]/span[2]/button[2]/span[2]'
        recs = driver.find_element_by_xpath(recs_xpath)
        if recs.is_displayed():
            recN = recs.text
            print(recN)
    except NoSuchElementException:        
        print("...")    

    #print(i,recs.text)
    ;
    
    
    

...
(8)
...
(4)
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
(1)
...
...
...
...
...
...


In [32]:
#Loop through all comments, taking title, content, and meta data (recs, author, etc)

comments_df = pd.DataFrame()

for i in range(1,n_comments+1):
    
    xpath = '//*[@id="comments"]/div[2]/div['+str(i)+']'
     
    comment_title = driver.find_element_by_xpath(xpath + '/div[1]')
    comment_body = driver.find_element_by_xpath(xpath + '/div[2]')
    comment_meta = driver.find_element_by_xpath(xpath + '/div[3]')
    # Grabbing author, comment recommendations
    #author_xpath = '/html/body/div[2]/main/article/div[5]/div[1]/section/div[2]/div['+str(i)+']/div[3]/span[1]/a[1]'
    #author = driver.find_element_by_xpath(author_xpath)
    
    # Recommendations number specifically doesn't work when there are none
    #recs_xpath = '/html/body/div[2]/main/article/div[5]/div[1]/section/div[2]/div['+str(i)+']/div[3]/span[2]/button[2]/span[2]'
        
    # Also grabbing the entire comments_action errors out on 10th iteration which has a div[4] instead of div[3]
    #recs_xpath = '/html/body/div[2]/main/article/div[5]/div[1]/section/div[2]/div['+str(i)+']/div[3]/span[2]'
    #recs = driver.find_element_by_xpath(recs_xpath)

    comment = pd.DataFrame([comment_title.text,comment_body.text,comment_meta.text])
    comments_df = pd.concat([comments_df,comment.T])
    
    ;
   

In [13]:
## Searching for comments by name returns empty lists

#recs_list = driver.find_elements_by_name('c-comments__actions')
recs_list = driver.find_elements_by_name('icon-rec c-comments__rec-link ') 
#recs_list = driver.find_elements_by_name('c-comments__post-actions u-clearfix')

type(recs_list)
print(recs_list)




[]
